# Cortex ML Classification SQL

This notebook will run an end-to-end ML classification task, utilising Snowflake's new Cortex ML powered functions. This notebook provides all code, as well as data used to train the classification model on.  

We will work through an example using data provided by the Irvine Machine Learning Repository; specifically their Bank Marketing Dataset to build a model to predict whether or not clients at a Portuguese bank subscribed to a term deposit during a marketing campaign.

In [ ]:
--Setup
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE COMPUTE_WH;
USE DATABASE CORTEX_ML_DEMO;
USE SCHEMA CLASSIFICATION_demo;

-- Create a csv file format to be used to ingest from the stage: 
CREATE OR REPLACE FILE FORMAT csv_ff
    TYPE = 'csv'
    SKIP_HEADER = 1,
    COMPRESSION = AUTO;
    
-- Create an external stage pointing to AWS S3 for loading our data:
CREATE OR REPLACE STAGE s3load 
    COMMENT = 'Quickstart S3 Stage Connection'
    URL = 's3://sfquickstarts/frostbyte_tastybytes/mlpf_quickstart/'
    FILE_FORMAT = csv_ff;

-- Define our table schema
CREATE OR REPLACE TABLE bank_marketing(
    AGE NUMBER,
    JOB TEXT, 
    MARITAL TEXT, 
    EDUCATION TEXT, 
    DEFAULT TEXT, 
    HOUSING TEXT, 
    LOAN TEXT, 
    CONTACT TEXT, 
    MONTH TEXT, 
    DAY_OF_WEEK TEXT, 
    DURATION NUMBER(4, 0), 
    CAMPAIGN NUMBER(2, 0), 
    PDAYS NUMBER(3, 0), 
    PREVIOUS NUMBER(1, 0), 
    POUTCOME TEXT, 
    EMPLOYEE_VARIATION_RATE NUMBER(2, 1), 
    CONSUMER_PRICE_INDEX NUMBER(5, 3), 
    CONSUMER_CONFIDENCE_INDEX NUMBER(3,1), 
    EURIBOR_3_MONTH_RATE NUMBER(4, 3),
    NUMBER_EMPLOYEES NUMBER(5, 1),
    CLIENT_SUBSCRIBED BOOLEAN);

-- Ingest data from S3 into our table:
COPY INTO bank_marketing
FROM @s3load/cortex_ml_classification.csv;

### View Data

In [ ]:
-- View a sample of the ingested data: 
SELECT * FROM bank_marketing LIMIT 100;

### Prepare Training & Inference Views

In [ ]:
-- Count of subscribed vs not subscribed: 
SELECT client_subscribed, COUNT(1) as num_rows
FROM bank_marketing
GROUP BY 1;

In [ ]:
-- Create a view with a column that will be filtered for either training/inference purposes
CREATE OR REPLACE VIEW partitioned_data as (
  SELECT *, 
        CASE WHEN UNIFORM(0::float, 1::float, RANDOM()) < .95 THEN 'training' ELSE 'inference' END AS split_group
  FROM bank_marketing
);

-- Training data view: 
CREATE OR REPLACE VIEW training_view AS (
  SELECT * EXCLUDE split_group
  FROM partitioned_data 
  WHERE split_group = 'training');

-- Inference data view
CREATE OR REPLACE VIEW inference_view AS (
  SELECT * EXCLUDE split_group
  FROM partitioned_data 
  WHERE split_group = 'inference');

### Train Classifier and Make Predictions

In [ ]:
-- Train our classifier: 
CREATE OR REPLACE snowflake.ml.classification bank_classifier(
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'training_view'),
    TARGET_COLNAME => 'CLIENT_SUBSCRIBED',
    CONFIG_OBJECT => {'evaluate': TRUE , 'on_error': 'skip'}
);


In [ ]:
-- Confirm model has been created
SHOW snowflake.ml.classification;

In [ ]:
-- Create the Predictions
SELECT bank_classifier!PREDICT(INPUT_DATA => object_construct(*))
    AS prediction FROM inference_view;

In [ ]:
CREATE OR REPLACE TABLE predictions AS (
SELECT 
    CLIENT_SUBSCRIBED,
    prediction:class::boolean as prediction, 
    prediction:probability:False as false_probability,
    prediction:probability:True as true_probability
FROM
    (
    SELECT bank_classifier!PREDICT(object_construct(*)) AS prediction, CLIENT_SUBSCRIBED
    FROM inference_view
    ));

SELECT *
FROM predictions
LIMIT 100;

### Evaluate Model

In [ ]:
-- Calculate the Confusion Matrix
CALL bank_classifier!SHOW_CONFUSION_MATRIX();

In [ ]:
-- Calculate the evaluation metrics
CALL bank_classifier!SHOW_EVALUATION_METRICS();

CALL bank_classifier!SHOW_GLOBAL_EVALUATION_METRICS();

In [ ]:
CALL bank_classifier!SHOW_THRESHOLD_METRICS();

In [ ]:
CALL bank_classifier!SHOW_FEATURE_IMPORTANCE();